In [1]:
from unsloth import FastLanguageModel

max_seq_length = 2048

dtype = None

load_in_4bit = True

# 모델 설정
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="./new_model_inst",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    cache_dir='/data'
)

Unsloth: Your Flash Attention 2 installation seems to be broken?
A possible explanation is you have a new CUDA version which isn't
yet compatible with FA2? Please file a ticket to Unsloth or FA2.
We shall now use Xformers instead, which does not have any performance hits!
We found this negligible impact by benchmarking on 1x A100.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-09-21 15:36:41.236873: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-21 15:36:41.262923: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-21 15:36:41.692250: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


==((====))==  Unsloth 2024.8: Fast Llama patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.66 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.0+cu121. CUDA = 8.6. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.24. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = True,
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.8 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [3]:
import pandas as pd
from datasets import Dataset

df = pd.read_csv('./news_summ.csv', encoding='UTF-8')

dataset = Dataset.from_pandas(df)

dataset

Dataset({
    features: ['news', 'summary'],
    num_rows: 42803
})

In [4]:
def formatting_summ_prompts_func(examples):
    output_texts = []
    for i in range(len(examples['news'])):
        messages = [
            {"role": "system", "content": "정확한 챗봇으로서 상대방의 입력에 대해 요약을 하자. 모든 대답은 한국어(Korean)으로 대답해줘."},
            {"role": "user", "content": "{}".format(examples['news'][i])},
            {"role": "assistant", "content": "{}".format(examples['summary'][i])}
        ]
        # LLAMA의 모델 클래스에 맞는 기본 채팅 템플릿 적용
        chat_message = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        output_texts.append(chat_message)

    return {"text": output_texts}

In [6]:
formatted_dataset = dataset.map(formatting_summ_prompts_func, batched=True)

train_test_split = formatted_dataset.train_test_split(test_size=0.2, seed=42)
train_dataset    = train_test_split['train']
eval_dataset     = train_test_split['test']

train_dataset

Map:   0%|          | 0/42803 [00:00<?, ? examples/s]

Dataset({
    features: ['news', 'summary', 'text'],
    num_rows: 34242
})

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=128,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=8,
        gradient_accumulation_steps=8,
        warmup_steps=100,
        max_steps=1000,
        learning_rate=3e-5,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=100,
        optim="adamw_torch",
        weight_decay=0.01,
        lr_scheduler_type="cosine",
        seed=3407,
        output_dir="outputs",
        save_steps=100,
    ),
)

trainer_stats = trainer.train()

Map (num_proc=128):   0%|          | 0/34242 [00:00<?, ? examples/s]

TimeoutError: 

In [ ]:
model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 61.48 out of 83.48 RAM for saving.


100%|██████████| 32/32 [00:01<00:00, 26.46it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at model into bf16 GGUF format.
The output location will be ./model/unsloth.BF16.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: model
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00004.safetensors'
INFO:hf-to-gguf:token_embd.weight,           torch.bfloat16 --> BF16, shape = {4096, 128256}
INFO:hf-to-gguf:blk.0.attn_norm.weight,      torch.bfloat

In [ ]:
import os
import shutil

source_path      = '/content/model'
destination_path = '/content/drive/MyDrive/new_model_inst'

if not os.path.exists(destination_path):
    os.makedirs(destination_path)

# source_path 경로에 있는 모든 파일과 폴더를 이동
for filename in os.listdir(source_path):
    file_path = os.path.join(source_path, filename)
    if os.path.isfile(file_path) or os.path.isdir(file_path):
        shutil.move(file_path, destination_path)

In [ ]:
source_path2      = '/content/outputs'
destination_path2 = '/content/drive/MyDrive/outputs'

if not os.path.exists(destination_path2):
    os.makedirs(destination_path2)

# source_path 경로에 있는 모든 파일과 폴더를 이동
for filename in os.listdir(source_path2):
    file_path = os.path.join(source_path2, filename)
    if os.path.isfile(file_path) or os.path.isdir(file_path):
        shutil.move(file_path, destination_path2)